In [1]:
import pandas as pd
import json

Open a single json and look at it

Bring in the data as a dictionary and then it is quite easy to read in as a Pandas to play around with

In [46]:
data = []
with open('stack_exchange_data/corpus/apple/0.json') as f:
    for line in f:
        data.append(json.loads(line))

In [47]:
data

[{u'body_text': u'External Disk to use with iTunes in a AirPort Extreme I have a external disk that I use now to have my iTunes collection, I want to connect the same disk to the AirPort Extreme so I dont have to use a USB port in my Notebook. \nDoes anyone have a reccomendation?\n',
  u'cluster_id': 0,
  u'novelty': True,
  u'order': 0,
  u'post_id': u'391'},
 {u'body_text': u'External Disk to use with TimeMachine in a AirPort Extreme Any one know any external disk that I can use to do this?\n',
  u'cluster_id': 0,
  u'order': 1,
  u'post_id': u'390'}]

In [48]:
pd_data = pd.DataFrame(data)

In [49]:
pd_data

,body_text,cluster_id,novelty,order,post_id
0,External Disk to use with iTunes in a AirPort ...,0,True,0,391
1,External Disk to use with TimeMachine in a Air...,0,NaN,1,390


In [ ]:
#bring in all the json files from a folder

In [64]:
import os

In [72]:
data_all = []
for filename in os.listdir('stack_exchange_data/corpus/apple/'):
    if filename.endswith('.json'):
        with open('stack_exchange_data/corpus/apple/' + filename) as f:
            for line in f:
                data_all.append(json.loads(line))

In [87]:
file_path = 'stack_exchange_data/corpus/apple/'

Put this all together to demo the code that is in utils/load_json_to_pandas.py

In [88]:
data_all = []
if file_path.endswith('.json'):
    with open(file_path) as f:
            for line in f:
                data_all.append(json.loads(line))
else:
    try:
        for filename in os.listdir(file_path):
            if filename.endswith('.json'):
                with open(file_path + filename) as f:
                    for line in f:
                        data_all.append(json.loads(line))
    except:
        print("Only *.json files or folders can be processed.  Error processing: " + file_path)
        pass

In [89]:
len(data_all)

5110

In [90]:
pd_data_all = pd.DataFrame(data_all)

In [91]:
pd_data_all.head()

,body_text,cluster_id,novelty,order,post_id
0,External Disk to use with iTunes in a AirPort ...,0,True,0,391
1,External Disk to use with TimeMachine in a Air...,0,False,1,390
2,Merge two accounts on iTunes? I just noticed t...,10,True,0,1594
3,Is it possible to merge two iTunes Store accou...,10,False,1,1460
4,Swap key €->$ permanently I have a Macbook Air...,100,True,0,8787


A handy webpage to see how to query data in Pandas is at: http://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html

In [41]:
#Look to see how many documents are in each cluster
pd_data_all.groupby("cluster_id").size()

cluster_id
0        2
4        5
7        2
8        3
9       11
10       2
11       1
12       1
14       1
17       2
18       2
21       1
22       2
23       6
24       4
27       2
29       2
32       5
38      15
41       3
43       3
44       2
46       7
48       3
50       4
51       8
57       4
59       5
62       3
67      11
        ..
4357     4
4371     2
4373     2
4375     1
4378     3
4379     2
4386     4
4392     2
4393     3
4394     2
4398     4
4415     3
4416    11
4417     2
4421     2
4423     2
4426     3
4429     2
4430     2
4438     2
4439     2
4453     3
4457     2
4458     2
4468     2
4470     2
4471     2
4474     2
4478     4
4494     2
dtype: int64

In [42]:
#so how many clusters are there (those with non-zero which this current dataset has)?
len(pd_data_all.groupby("cluster_id").size())

1350

In [28]:
#We are going to try to bring in the data into Spark.
#If you aren't running Spark or have PySpark this part will not work...
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SQLContext 

try:
    sc = SparkContext()
except:
    sc = SparkContext._active_spark_context

sqlCtx = SQLContext(sc)

In [29]:
data1 = sqlCtx.read.json("stack_exchange_data/corpus/apple/921.json")

In [30]:
data1.collect()

[Row(body_text=u"Upgrade to higher memory or faster processor on MacBook Air I'm zeroing on buying the 2012 MacBook Air. My usage of the machine will be mostly for development work (MySQL, Eclipse, Xcode, Android, etc.). I will also be running Windows 7 with SQL Server in a virtual box or by using Boot Camp. My question is which upgrade will result in the most bang for the buck: more RAM or a faster processor? Does putting higher memory/processor make the machine run hotter?\n", cluster_id=921, novelty=True, order=0, post_id=u'55074')]

In [31]:
data1.registerTempTable("stack")
sqlCtx.sql("select * from stack where order=0").take(1)

[Row(body_text=u"Upgrade to higher memory or faster processor on MacBook Air I'm zeroing on buying the 2012 MacBook Air. My usage of the machine will be mostly for development work (MySQL, Eclipse, Xcode, Android, etc.). I will also be running Windows 7 with SQL Server in a virtual box or by using Boot Camp. My question is which upgrade will result in the most bang for the buck: more RAM or a faster processor? Does putting higher memory/processor make the machine run hotter?\n", cluster_id=921, novelty=True, order=0, post_id=u'55074')]

In [32]:
data1.printSchema()

root
 |-- body_text: string (nullable = true)
 |-- cluster_id: long (nullable = true)
 |-- novelty: boolean (nullable = true)
 |-- order: long (nullable = true)
 |-- post_id: string (nullable = true)



In [ ]:
#very similar things that I did above can be done in Spark